# Доклад по Scrapy 
Выполнил Хоранян Нарек ЭАД

### Scrapy? Что это вообще?

Процитируем документацию:
>Scrapy (/ˈskreɪpaɪ/) is an application framework for crawling web sites and extracting structured data which can be used for a wide range of useful applications, like data mining, information processing or historical archival.

>Even though Scrapy was originally designed for web scraping, it can also be used to extract data using APIs (such as Amazon Associates Web Services) or as a general purpose web crawler.


### Переведем с технического на человческий:
Scrapy - это ФРЕЙМВОРК, который позволяют ползти по сайтам и извлекать структурированные данные типа таблиц и вообще чего угодно что написано нормальным хтмлевым языком, то есть мега удобный парсер

### Почему Scrapy а не пацанские самопалы через request/aiohttp?
1. В основе Scrapy лежит *асинхронный* движок
2. Поддержка почти чего угодно:
   1. CSS-селекторы
   2. Генерация экспортов в множестве формаотв (JSON, CSV, XML) и их хранение в разных бэкендах (FTP, S3, локально)
   3. Обширная поддержка кодировок и автообнаружений для работу с иностранными нестандартными языками
   4. Модульность (через сигналы и грамотно описанное АПИ)(middlewares, extensions, and pipelines).
   5. Встроенные расширения и плагины для:
      1. cookies and session handling
      2. HTTP features like compression, authentication, caching
      3. user-agent spoofing
      4. robots.txt
      5. crawl depth restriction
3. Интеграция с Selenium через Scrapy-Selenium


## А, ну и пример

Сложнейший выбор сделают лишь сильнейшие. Поэтому мы сделаем самый маскулинный и dominance asserting мув который можно представить - запарсим Евраз.Маркет и по пути парсинга расскажу какие приколы вообще лежат в Scrapy

Почему Евраз.Маркет? Потому что изначально я хотел запарсить Северсталь.Маркет? А потому что придется поиграться с селениумом а мне хочется поиграться с селениумом (у челов слишком много уровней защиты), и мы будем использовать не просто Scrapy, а Scrapy-Selenium. Почему я не стал парсить Северсталь.Маркет? Потому что у северстали слишком сильная защита от ботов, для нее нужно использовать всю силу Selenium. Scrapy не позволяет этого сделать (потому что он выбрасывает свой драйвер, а не тот, что нам нужен), а взрывать исходный код библиотеки, чтобы селениум нельзя было задетектить не хочется

# Теперь уже само дело:

Код, к сожалению, сложно интегрировать в ноутбук, потому что под Scrapy-парсер требуется целый проект, поэтому описание будет в виде 
```python
# имя файла
<код>
```

[ссылка на сам проект](https://github.com/wolfrahmmetall/scrapy-report)

Корневой (~) папкой считаем саму основную папку `report`, в которой содержится `report`, в которой содержится еще один `report` (Scrapy это нужно)

1. ## Интеграция Scrapy-selenium
   Прописываем в терминале:
    ```sh
    report $ scrapy startproject report
    ```
    scrapy создаст нам новый проект:
    ```
    report
    │   ├── __init__.py
    │   ├── __pycache__
    │   │   ├── __init__.cpython-311.pyc
    │   │   └── settings.cpython-311.pyc
    │   ├── items.py
    │   ├── middlewares.py
    │   ├── pipelines.py
    │   ├── settings.py
    │   └── spiders
    └── scrapy.cfg
    ```
    В котором уже будут сгенерированы все нужные файлы

    Также в корневую директорию добавим Chromedriver  
    Зачем? Затем, что мы работаем на Chrome (Version 130.0.6723.117, arm64) (Firefox круче, но проверить кто-то должен) 

    Также есть проблема с тем, что Scrapy-selenium нативно не поддерживает `Selemium4`. Но есть обход: https://github.com/clemfromspace/scrapy-selenium/blob/5c3fe7b43ab336349ef5fdafe39fc87f6a8a8c34/scrapy_selenium/middlewares.py  
    засовываем его в `middlewares.py` по `Location` из `python -m pip show scrapy-selenium`

    Далее заходим в `settings.py` нашего проекта
    и прописываем (заменяем) следующее:
    ```python
    DOWNLOADER_MIDDLEWARES = {
   "scrapy_selenium.SeleniumMiddleware": 800
   }
   DEFAULT_REQUEST_HEADERS = {
   "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
   "Accept-Language": "ru",
   "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36" # Северсталь банит всех роботов, работают только ЮА
   }

   SELENIUM_DRIVER_ARGUMENTS = ["--headless=new"] 
    ```

   и о счастье, все должно работать

2. ## Пауки
   Самая главная фича Scrapy -- по сути парсер бот, который способен через движок Scrapy самостоятельно отправлять запросы и парсить сайты

   Scrapy умеет сам их генерировать с помощью шаблонов с помощью команды в терминале:
   ```sh
   report $ scrapy genspider spider name url
   ```
   он создаст папку `spiders` и в ней паука 

   нам нужен: 
   ```sh
   report $ scrapy genspider armatura https://market.severstal.com/ru/ru/o/c/46/96
   ```

   и Scrapy нам создал
   ```python
   #report/report/spiders/armatura.py
   import scrapy

   class ArmaturaSpider(scrapy.Spider):
      name = "armatura"
      allowed_domains = ["market.severstal.com"]
      start_urls = ["https://market.severstal.com/ru/ru/o/c/46/96"]

      def parse(self, response):
         pass
   ```

Т.к. мы парсим с селениумом, то `parse` выглядит так:

```python

    def start_requests(self):
        url = "https://evraz.market/metalloprokat/armatura/armatura_riflenaya/"
        yield SeleniumRequest(url=url, callback=self.parse)
        url = "https://evraz.market/metalloprokat/armatura/armatura_riflenaya/PAGEN_4=2"
        yield SeleniumRequest(url=url, callback=self.parse)

    def parse(self, response: scrapy.http.Response):
        driver: Chrome = response.request.meta['driver']
        products = driver.find_element(By.CSS_SELECTOR, "tbody")
        for product in products.find_elements(By.CSS_SELECTOR, '.catalog-item'):
            arm_name = product.find_element(By.CSS_SELECTOR, 'span.underline-stroke-link').find_element(By.CSS_SELECTOR, 'span').text
            length = product.find_element(By.CSS_SELECTOR, '.main-info__question').text.split(' ')[-1]
            left = product.find_element(By.CSS_SELECTOR, '.catalog-page-stock-status-block').text
            price = [prod.text for prod in product.find_elements(By.CSS_SELECTOR, '.text-nowrap')[:2]]
            yield {
                'name': arm_name,
                'length': length,
                'remaining': left,
                'price range': price
            }
```


Тогда, прогнав в терминале
```sh
~$ scrapy crawl <имя паука> *keys
```
можно запустить скрэпи движок и заставить бот работать

Можно прописать `-o` и вывести все данные из паука в `csv` или `json` файл и `-L` для установки выводимого в терминал уровня логгирования

## Как это работает?
При запуске движка Scrapy сначала прогоняет свой метод `start_requests`, `yield`'я реквесты по ссылкам. `start_request` может как просто использовать ссылки из поля `start_urls`, так и может быть переопределен.  
Запросы в `start_requests` могут быть как просто `scrapy.http.Request`, так и `SeleniumRequest` из `Scrapy-selenium`, который я использовал. Суть одна, ему нужен callback -- функция обработчик, который возвращает данные в удобном формате, чтобы их можно было передать в json или csv файл. Почему `yield`? Потому что движок асинхронный.

При этом нужно обратить внимание на одну вещь -- в `parse` передается аргумент `response: scrapy.http.Response`. Это очень важно, потому что в нем содержится вся информация о сайте, в частности html-разметка, а значит ее можно передать в `webdriver` и спарсить

### <font color='ff0000;'> Но это же не все пауки</font>

Scrapy дает еще несколько шаблонов пауков, но они не очень полезны в примере, поэтому приведу примеры с документации:

### <font color='ffff00'> Ползущий паук</font>

```python
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor


class MySpider(CrawlSpider):
    name = "example.com"
    allowed_domains = ["example.com"]
    start_urls = ["http://www.example.com"]

    rules = (
        # Extract links matching 'category.php' (but not matching 'subsection.php')
        # and follow links from them (since no callback means follow=True by default).
        Rule(LinkExtractor(allow=(r"category\.php",), deny=(r"subsection\.php",))),
        # Extract links matching 'item.php' and parse them with the spider's method parse_item
        Rule(LinkExtractor(allow=(r"item\.php",)), callback="parse_item"),
    )

    def parse_item(self, response):
        self.logger.info("Hi, this is an item page! %s", response.url)
        item = scrapy.Item()
        item["id"] = response.xpath('//td[@id="item_id"]/text()').re(r"ID: (\d+)")
        item["name"] = response.xpath('//td[@id="item_name"]/text()').get()
        item["description"] = response.xpath(
            '//td[@id="item_description"]/text()'
        ).get()
        item["link_text"] = response.meta["link_text"]
        url = response.xpath('//td[@id="additional_data"]/@href').get()
        return response.follow(
            url, self.parse_additional_page, cb_kwargs=dict(item=item)
        )

    def parse_additional_page(self, response, item):
        item["additional_data"] = response.xpath(
            '//p[@id="additional_data"]/text()'
        ).get()
        return item
```

Зачем он нужен? Он умеет переходить по ссылкам по определенным *правилам* -- то есть потенциально мы можем рекурсивно запарсить весь сайт с одной страницы.  
При этом сами правила -- callback:  
```python
class scrapy.spiders.Rule(link_extractor: LinkExtractor | None = None, callback: CallbackT | str | None = None, cb_kwargs: dict[str, Any] | None = None, follow: bool | None = None, process_links: ProcessLinksT | str | None = None, process_request: ProcessRequestT | str | None = None, errback: Callable[[Failure], Any] | str | None = None)[source]
```

Как видим, мы используем несколько `callback`ов для обработки страниц по ссылкам, по сути, аналогов `parse` и так постепенно спарсим все дерево

### <font color='ffff00'>XMLFeedSpider и CSVFeedSpider</font>

```python
from scrapy.spiders import XMLFeedSpider
from myproject.items import TestItem


class MySpider(XMLFeedSpider):
    name = "example.com"
    allowed_domains = ["example.com"]
    start_urls = ["http://www.example.com/feed.xml"]
    iterator = "iternodes"  # This is actually unnecessary, since it's the default value
    itertag = "item"

    def parse_node(self, response, node):
        self.logger.info(
            "Hi, this is a <%s> node!: %s", self.itertag, "".join(node.getall())
        )

        item = TestItem()
        item["id"] = node.xpath("@id").get()
        item["name"] = node.xpath("name").get()
        item["description"] = node.xpath("description").get()
        return item
```

Удобный паук, чтобы парсить `XML` и `CSV` файлы, он говорит обо всем за себя

## Только пауки?

Тоже нет. Есть класс `Item` и его наследники. Которые позволяют создать наш возвращаемый тип вида json-словаря с помощью полей типа `Field()` (честно? юзлесс)

## А почему именно Scrapy?
Валидный вопрос. С Selenium можно спарсить страшные вещи вроде Северсталь.Маркета. А для простых сайтов хватает и просто requests. НО Scrapy обладает гораздо более тонкой настройкой.
Можно установить Скачивающих посредников, аргументы для селениума и много чего другого. К тому же не надо возиться с драйверами в селениумом -- тут все уже готово

В репозитории можно посмотреть на примеры парсеров для разных страниц и баш скрипт, который запускает их вместе. Там и многостраничный парсинг (отладить я его не успел, потому что превысил лимит запросов, даже с рандомными юзер агентами), и уровни логгировния, и многое прочее

### Рандомные юзер агенты? А что это
Юзер Агент --- программа, способная имитировать человека при использовании сайтов. Но проблема в том, что сайт способен обнаружить на каком IP+данных юзер агента бот находится, поэтому банит его.
Но в то же время, Scrapy с помощью `scrapy_user_agents` (тоже модуль, скачивается из пипа) предоставляет возможность рандомно подставлять рандомного юзер агента. Соответственно, сайту будет сложнее обнаружить нас